In [2]:
from __future__ import annotations
from typing import Tuple, List, Dict

In [ ]:
def re_parse(): raise NotImplementedError
def parse_node(): raise NotImplementedError

In [ ]:
class Node:
    def __init__(self, kind: str, *args):
        self.kind = kind
        self.args = args
    def __repr__(self):
        return f'Node({self.kind}, {", ".join(repr(a) for a in self.args)})'

def parse_split(r: str, idx: int) -> Tuple[int, Node]:
    """Takes subexpressions from parse_concat and parses them into a split node"""
    idx, prev = parse_concat(r, idx)
    while idx < len(r):
        if r[idx] == ')':
            # return to the outer parse node
            break
        assert r[idx] == '|', f'Expected "|" got {r[idx]}'
        idx, node = parse_concat(r, idx + 1)
        prev = ('split', prev, node)
    return idx, prev

def parse_concat(r: str, idx: int) -> Tuple[int, Node]:
    """Takes subexpressions from parse_node and parses them into a concat node"""
    prev = None
    while idx < len(r):
        if r[idx] in '|)':
            # return to the outer parse_split or parse_node
            break
        idx, node = parse_node(r, idx) # parse a node
        if prev is None:
            prev = node
        else:
            prev = ('cat', prev, node)
    return idx, prev

def parse_node(r: str, idx: int) -> Tuple[int, Node]:
    """Parses a single node"""
    ch = r[idx]
    idx += 1
    assert ch not in '|)', f'Expected a character, got {ch}'
    if ch == '(':
        idx, node = parse_split(r, idx)
        if idx < len(r) and r[idx] == ')':
            idx += 1
        else:
            raise SyntaxError('Expected ")": unbalanced parentheses')
    elif ch == '.':
        node = ('dot',)
    elif ch in '*+{':
        raise Exception('nothing to repeat')
    else:
        node = ('char', ch)

    idx, node = parse_postfix(r, idx, node)
    return idx, node

def parse_postfix(r: str, idx: int, node: Node) -> Tuple[int, Node]:
    if idx == len(r) or r[idx] not in '*+{':
        return idx, node
    
    ch = r[idx]
    idx += 1

    if ch = 

In [ ]:
def test():
    assert re_parse('') is None
    assert re_parse('.') == 'dot'
    assert re_parse('a') == 'a'
    assert re_parse('ab') == ('cat', 'a', 'b')
    assert re_parse('a|b') == ('split', 'a', 'b')
    assert re_parse('a+') == ('repeat', 'a', 1, float('inf'))
    assert re_parse('a{3,6}') == ('repeat', 'a', 3, 6)
    assert re_parse('a|bc') == ('split', 'a', ('cat', 'b', 'c'))